In [ ]:
"""
[クラスタリング]
物事を似た者同士が同じグループになるように分類するプロセスとして定義される.
もっともよく使われる教師なし学習の技術は, クラスタリングである.

クラスタリングは1つの最適化問題である.
ある制約のもとである目的関数を最適化するクラスタの集合を求めることがゴールである.
2つの標本がどれだけ近いかを表す距離という計量が与えられているとき, 同じクラスタに属している標本間の距離を最小化する.
つまりクラスタ内の標本間の創意性を最小化するように目的関数を設定する必要がある.
"""

In [ ]:
"""
[変動性]
クラスタ内の標本同士がどれくらい異なっているか、という分散に似た基準として、変動性と呼ばれるものがある.

変動制は,
variability(c) = sum(distance(mean(c), e)**2)
で定義される.

mean(c)は, クラスタc内の全てのサンプルの特徴ベクトルの平均である. ベクトルの平均とは, 成分ごとに平均を計算したものである.

単一のクラスタでの分散の定義は, 次のようにクラスタ集合Cの相違性を定義するのに拡張される.

dissimilarity(C) = sum(variability(c))

変動性をクラスタのサイズで割っていないため, サイズが大きいほど相違性が増えることに注意.
そういう設計である.

"""